# Rips coplex examples

Adapted from:
- https://gudhi.inria.fr/python/latest/rips_complex_user.html

In [ ]:
import gudhi
import numpy as np

#### Example with point cloud

In [ ]:
rips_complex = gd.RipsComplex(points=[[1, 1], [7, 0], [4, 6], [9, 6], [0, 14], [2, 19], [9, 17]],
                                 max_edge_length=12.0)

simplex_tree = rips_complex.create_simplex_tree(max_dimension=1)
result_str = 'Rips complex is of dimension ' + repr(simplex_tree.dimension()) + ' - ' + \
    repr(simplex_tree.num_simplices()) + ' simplices - ' + \
    repr(simplex_tree.num_vertices()) + ' vertices.'
print(result_str)
fmt = '%s -> %.2f'
for filtered_value in simplex_tree.get_filtration():
    print(fmt % tuple(filtered_value))

#### Example with correlation

In [ ]:


# User defined correlation matrix is:
# |1     0.06    0.23    0.01    0.89|
# |0.06  1       0.74    0.01    0.61|
# |0.23  0.74    1       0.72    0.03|
# |0.01  0.01    0.72    1       0.7 |
# |0.89  0.61    0.03    0.7     1   |
correlation_matrix=np.array([[1., 0.06, 0.23, 0.01, 0.89],
                            [0.06, 1., 0.74, 0.01, 0.61],
                            [0.23, 0.74, 1., 0.72, 0.03],
                            [0.01, 0.01, 0.72, 1., 0.7],
                            [0.89, 0.61, 0.03, 0.7, 1.]], float)

distance_matrix = 1 - correlation_matrix
rips_complex = gudhi.RipsComplex(distance_matrix=distance_matrix, max_edge_length=1.0)

simplex_tree = rips_complex.create_simplex_tree(max_dimension=1)
result_str = 'Rips complex is of dimension ' + repr(simplex_tree.dimension()) + ' - ' + \
    repr(simplex_tree.num_simplices()) + ' simplices - ' + \
    repr(simplex_tree.num_vertices()) + ' vertices.'
print(result_str)
fmt = '%s -> %.2f'
for filtered_value in simplex_tree.get_filtration():
    print(fmt % tuple(filtered_value))

# Apply Gudhi to  geometric matrix

In [ ]:
import numpy as np
import math
import gudhi as gd
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
from sklearn import metrics

### Get ordered geometric matrix
The notebook is shipped with files for N={40, 80, 160} and cloud_dim={20}

In [ ]:
from numpy import genfromtxt

N = 40
cloud_dim = 20
ord_mat = genfromtxt('ordering_matrix_points{}_dims{}.csv'.format(N, cloud_dim), delimiter=',')

max_ord_val = np.amax(ord_mat)

### Compute topology

In [ ]:
rips_complex = gd.RipsComplex(distance_matrix=ord_mat, max_edge_length=max_ord_val)

simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)

#### Get Betti curve

In [ ]:
def get_betti_curves(complex_object, source_matrix):
    betti_min_value = np.amin(source_matrix).astype(int)
    betti_max_value = np.amax(source_matrix).astype(int)
    
    persistence = complex_object.persistence()
    
    matrix_range_values = range(betti_min_value,betti_max_value-1)
    betti_values = complex_object.persistent_betti_numbers(from_value=betti_max_value, to_value=betti_min_value)
    betti_curves_matrix = np.zeros((len(betti_values), len(matrix_range_values)), dtype=int)

    for step in matrix_range_values:
        betti_val = complex_object.persistent_betti_numbers(from_value=step+1, to_value=step)
        betti_curves_matrix[:,step] = betti_val
    
    return betti_curves_matrix, matrix_range_values
        
def plot_betti_curves(betti_curves_matrix, matrix_range_values, min_dim=1):
    x_vals = np.array(image_range_values)
    max_betti = betti_curves_matrix.shape[0]
    
    for betti in range(min_dim,max_betti):
        plt.plot(x_vals, betti_curves_matrix[betti,:])
    plt.ylabel("# cycles")
    plt.xlabel("step")
    plt.title("Betti curves")


In [ ]:
betti_curves_matrix, image_range_values = get_betti_curves(simplex_tree, ord_mat)
plot_betti_curves(betti_curves_matrix, image_range_values)